# Milestone 1.1 -- Captstone Proposal Addon
Here, we do the work to get the Astral package working to predict sunlight status for the times of the traffic violations.

In [1]:
# Define the File path for the traffic data
TRAFFIC_DATA = 'KenjiOman-Traffic_Violations.csv.gz'

In [2]:
# Import Modules
import pandas as pd
import astral

Note, although we have latitutde and longitude data for most all the traffic violations, the entirety of Montgomery County is right next to Washington DC. Since Washington DC comes with the astral package, let's just predict our daylight status based on DC, to make things simpler.  So, we only need the date and time information from the traffic violations dataset

In [3]:
# Grab the relevant data
df = pd.read_csv(TRAFFIC_DATA, header=0, usecols=['Date Of Stop', 'Time Of Stop'],
                 compression='gzip', infer_datetime_format=True,
                 parse_dates=[['Date Of Stop', 'Time Of Stop']])
# Rename the column
df.columns = ['DateTime']
df.head()

,DateTime
0,2013-09-24 17:11:00
1,2017-08-29 10:19:00
2,2014-12-01 12:52:00
3,2017-08-29 09:22:00
4,2017-08-28 23:41:00


In [4]:
# Let's add back columns for the date and time
df['date'] = df.DateTime.dt.date
df['time'] = df.DateTime.dt.time
print(df.shape)
df.head()

(1292399, 3)


,DateTime,date,time
0,2013-09-24 17:11:00,2013-09-24,17:11:00
1,2017-08-29 10:19:00,2017-08-29,10:19:00
2,2014-12-01 12:52:00,2014-12-01,12:52:00
3,2017-08-29 09:22:00,2017-08-29,09:22:00
4,2017-08-28 23:41:00,2017-08-28,23:41:00


In [5]:
# set up astral's city to be DC
city = astral.Astral()['Washington DC']

In [6]:
# Grab all the unique dates
dates = df.date.unique()

In [7]:
# Create a function for, given all the dates, grab relevant times for different solar events
def grab_sun_info(dates=dates, city=city):
    # Set up results dictionary to store results. These definitions from the Astral
    # documentation page, https://astral.readthedocs.io/en/latest/
    results = {
        # The date in question
        'date': [],
        # Dawn: time when sun is 6 degrees below horizon, before sunrise
        #   Also, the beginning of the morning blue hour
        'dawn': [],
        # Sunrise: Time when top of sun breaks horizon
        'sunrise': [],
        # Sunset: Time when sun is about to disapear below the horizon
        'sunset': [],
        # Dusk: Time when sun is 6 degrees below horizon, after sunset
        #   Also, the end of the evening blue hour
        'dusk': [],
        # Begining of morning golden hour (also, end of morning blue hour),
        #   Time when sun is 4 degrees below horizon, before sunrize
        'morning_golden_start': [],
        # End of morning golden hour, time when sun is 6 degrees
        # above horizon, right after sunrise
        'morning_golden_end': [],
        # Begining of evening golden hour, when sun is 6 degrees above
        # horizon, right before sunset
        'evening_golden_start': [],
        # End of evening golden hour, when sun is 4 degrees below horizon,
        # right after sunset
        'evening_golden_end': [],
        # And the begining of the morning blue
        'morning_blue_start': [],
        # And the end of the morning blue
        'morning_blue_end': [],
        # And the beginning of the evening blue
        'evening_blue_start': [],
        # and the end of the evening blue
        'evening_blue_end': []
    }
    
    # Now, go through each of the dates
    for date in dates:
        # store the date
        results['date'].append(date)
        
        # Calculate and store Dawn, sunrise, sunset, dusk values
        results['dawn'].append(city.dawn(date))
        results['sunrise'].append(city.sunrise(date))
        results['sunset'].append(city.sunset(date))
        results['dusk'].append(city.dusk(date))
        
        # Now, grab the Start/ end of the morning golden hour, and save
        golden = city.golden_hour(direction=astral.SUN_RISING, date=date)
        results['morning_golden_start'] = golden[0]
        results['morning_golden_end'] = golden[1]
        
        # And the start/ end of the evening golden hour
        golden = city.golden_hour(direction=astral.SUN_SETTING, date=date)
        results['evening_golden_start'] = golden[0]
        results['evening_golden_end'] = golden[1]
        
        # Now, grab the Start/end of the morning blue hour, and save
        blue = city.blue_hour(direction=astral.SUN_RISING, date=date)
        results['morning_blue_start'] = blue[0]
        results['morning_blue_end'] = blue[1]
        
        # And the start/end of the evening blue hour
        # Now, grab the Start/end of the morning blue hour, and save
        blue = city.blue_hour(direction=astral.SUN_SETTING, date=date)
        results['evening_blue_start'] = blue[0]
        results['evening_blue_end'] = blue[1]
        
    # Convert the results to a pd.DataFrame, and return
    return pd.DataFrame(results)

In [8]:
# Now, get the sun info
sun_info = grab_sun_info(dates, city)
# And convert all the times to their naive counterparts,
# and just their times
for col in sun_info.drop(columns=['date']).columns:
    sun_info[col] = sun_info[col].dt.tz_localize(None)
    sun_info[col] = sun_info[col].dt.time
sun_info.head()

,date,dawn,sunrise,sunset,dusk,morning_golden_start,morning_golden_end,evening_golden_start,evening_golden_end,morning_blue_start,morning_blue_end,evening_blue_start,evening_blue_end
0,2013-09-24,06:30:12,06:57:25,19:03:09,19:30:22,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
1,2017-08-29,06:04:03,06:32:16,19:46:03,20:14:15,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
2,2014-12-01,06:41:27,07:11:41,16:42:13,17:12:28,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
3,2017-08-28,06:03:01,06:31:18,19:47:36,20:15:53,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
4,2013-08-27,06:01:57,06:30:18,19:49:12,20:17:32,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18


In [9]:
# Found there is some discrpancy between dawn and morning_blue_start,
# and dusk and evening_blue_end, so let's just use the numbers
# from morning/evening blue. Sow, drop dawn, sunrise, sunset, and dusk values
sun_info.drop(columns=['dawn', 'sunrise', 'sunset', 'dusk'], inplace=True, errors='ignore')
sun_info.head()

,date,morning_golden_start,morning_golden_end,evening_golden_start,evening_golden_end,morning_blue_start,morning_blue_end,evening_blue_start,evening_blue_end
0,2013-09-24,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
1,2017-08-29,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
2,2014-12-01,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
3,2017-08-28,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
4,2013-08-27,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18


In [10]:
# Join this to the original dataframe
df = df.merge(sun_info, on='date', how='left')
df.head()

,DateTime,date,time,morning_golden_start,morning_golden_end,evening_golden_start,evening_golden_end,morning_blue_start,morning_blue_end,evening_blue_start,evening_blue_end
0,2013-09-24 17:11:00,2013-09-24,17:11:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
1,2017-08-29 10:19:00,2017-08-29,10:19:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
2,2014-12-01 12:52:00,2014-12-01,12:52:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
3,2017-08-29 09:22:00,2017-08-29,09:22:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18
4,2017-08-28 23:41:00,2017-08-28,23:41:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18


In [11]:
# Now, define the daylight status for the time of every row
# If this row appears before morning_blue_start, or after evening_blue_end,
# it is night time
df.loc[(df.time < df.morning_blue_start) | (df.time > df.evening_blue_end), 'daylight'] = 'night'
    
# Otherwise, if it is in the morning blue time
df.loc[(df.time >= df.morning_blue_start) & (df.time < df.morning_blue_end), 'daylight'] = 'blue_hour'
# or if it is in the evening blue hour
df.loc[(df.time > df.evening_blue_start) & (df.time <= df.evening_blue_end), 'daylight'] = 'blue_hour'
    
# Or, if in the morning golden hour
df.loc[(df.time >= df.morning_golden_start) & (df.time < df.morning_golden_end), 'daylight'] = 'golden_hour'
# or if it is in the evening golden hour
df.loc[(df.time > df.evening_golden_start) & (df.time <= df.evening_golden_end), 'daylight'] = 'golden_hour'
    
df.loc[df.daylight.isna(), 'daylight'] = 'day'

df.head()

,DateTime,date,time,morning_golden_start,morning_golden_end,evening_golden_start,evening_golden_end,morning_blue_start,morning_blue_end,evening_blue_start,evening_blue_end,daylight
0,2013-09-24 17:11:00,2013-09-24,17:11:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18,day
1,2017-08-29 10:19:00,2017-08-29,10:19:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18,day
2,2014-12-01 12:52:00,2014-12-01,12:52:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18,day
3,2017-08-29 09:22:00,2017-08-29,09:22:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18,day
4,2017-08-28 23:41:00,2017-08-28,23:41:00,06:10:00,07:04:15,19:16:54,20:11:09,05:58:51,06:10:00,20:11:09,20:22:18,night


In [12]:
# Check how many date/time duplicates there are
df.shape, df.drop_duplicates().shape

((1292399, 12), (626010, 12))

In [13]:
# Looks like there were duplicate date/times in the data, so let's just drop them
df.drop_duplicates(inplace=True)

In [14]:
# Check we don't have duplicate date/times
df.DateTime.value_counts().head()

2012-07-27 13:58:00    1
2013-10-04 14:14:00    1
2017-02-05 10:35:00    1
2013-03-29 05:36:00    1
2013-04-26 07:58:00    1
Name: DateTime, dtype: int64

In [15]:
# Every date/time is now unique,
# let's check how much of each daylight value we have
df.daylight.value_counts()

day            362803
night          225971
golden_hour     31604
blue_hour        5632
Name: daylight, dtype: int64

In [16]:
# Export the data
df[['DateTime', 'daylight']].to_csv('KenjiOman-M01_daylight.csv.gz', index=False, compression='gzip')